In [2]:
import torch
from torchvision import models
import torch.nn as nn

# 获取alexNet预训练参数
# alexNet = models.alexnet(pretrained=True)
# torch.save(alexNet.state_dict(),'alexNet_weights.pth')
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

 
Using cpu device


In [3]:
from collections import abc
"""修改 alexnet 使其 iterable"""
class AlexNet(nn.Module):
    def __init__(self,input_layer = 3,num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_layer, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

    def __iter__(self,):
        return SentenceIterator(self.features,self.avgpool,self.classifier)

class SentenceIterator(abc.Iterator):
    def __init__(self,features,avgpool,classifier):
        self.features = features
        self.avg_pool = avgpool
        self.classifier = classifier
        self._index = 0
        self.len1 = len(features)
        self.len2 = 1
        self.len3 = len(classifier)


    def __next__(self):
        try:
            if self._index < self.len1:
                layer = self.features[self._index]
            elif self._index < (self.len1 + self.len2):
                layer = self.avg_pool
            else:
                layer = self.classifier[self._index - self.len1 - self.len2]
        except IndexError:
            raise StopIteration()
        else:
            self._index += 1

        return layer

In [4]:
def alexnet(pretrained: bool = False, **kwargs) -> AlexNet:
    model = AlexNet(**kwargs).to(device)
    if pretrained:
        state_dict = torch.load('./00_model_parameters/alexNet_weights.pth')
        model.load_state_dict(state_dict)
    return model

In [5]:
"""
    使用与训练好的参数先预测一波fashionmnis图像分类的准确率
"""
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

resize = 224
trans = transforms.Compose([
    transforms.Resize(resize),
    transforms.ToTensor(),
])
training_data = datasets.FashionMNIST(root = "../data",train=True,download=False,transform=trans)
test_data = datasets.FashionMNIST(root = "../data",train=False,download=True,transform=trans)

mytest_data = []
for idx,(x,y) in enumerate(test_data):
    mytest_data.append((torch.cat((x,x,x),dim=0),y))

train_dataloader = DataLoader(training_data,batch_size=64,shuffle=True)
test_dataloader = DataLoader(mytest_data,batch_size=64,shuffle=True)

In [6]:
"""
    测定test数据集准确率
    batch input [64,1,224,224]
    batch label [64]

    batch output [64,10]
"""
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct,all = 0,0,0
    with torch.no_grad():
        for idx,(X,y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            all += 64
            # X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            if(all % 640 == 0):
                print(f"all number: {all}   correct number: {correct}")
                print(f"Accuracy: {(100*(correct/all)) :> 0.1f}% , avg loss :{test_loss/(idx+1):>8f}")
                print(f"--------------------------------------------------")
    test_loss /= num_batches
    correct /= size
    print(f"test error :\n Accuracy: {(100*correct) :> 0.1f} , avg loss :{test_loss:>8f}")

# 预训练最后输出维度1000 所以目前fashion数据集只能多加个 linear 1000->10
# 后续换成cifar-10 数据集可以正常运行
alexNet = nn.Sequential(
    alexnet(pretrained=False,num_classes=1000),
    nn.Linear(1000,10),
).to(device)
test(test_dataloader,alexNet,nn.CrossEntropyLoss())

all number: 640   correct number: 69.0
Accuracy:  10.8% , avg loss :2.302552
--------------------------------------------------
all number: 1280   correct number: 132.0
Accuracy:  10.3% , avg loss :2.302895
--------------------------------------------------
all number: 1920   correct number: 198.0
Accuracy:  10.3% , avg loss :2.302572
--------------------------------------------------
all number: 2560   correct number: 259.0
Accuracy:  10.1% , avg loss :2.302608
--------------------------------------------------
all number: 3200   correct number: 316.0
Accuracy:  9.9% , avg loss :2.302572
--------------------------------------------------
all number: 3840   correct number: 385.0
Accuracy:  10.0% , avg loss :2.302678
--------------------------------------------------
all number: 4480   correct number: 447.0
Accuracy:  10.0% , avg loss :2.302655
--------------------------------------------------
all number: 5120   correct number: 507.0
Accuracy:  9.9% , avg loss :2.302620
---------------

In [7]:
"""
    尝试分解每层的计算
"""
import time
x = torch.rand(size=(10000,3,224,224))
x = x.to(device)
alexNet2 = alexnet(input_layer = 3,num_classes = 1000)
print(next(alexNet2.parameters()).device)

for layer in alexNet2:
    start_time = int(round(time.time() * 1000))
    x = layer(x)
    end_time = int(round(time.time() * 1000))
    # print(x.device)
    print(layer.__class__.__name__,'output shape:\t',x.shape,
          f'\t computation time: {(end_time - start_time)/1000 :>3f} s')

cpu
Conv2d output shape:	 torch.Size([10000, 64, 55, 55]) 	 computation time: 31.926000 s
ReLU output shape:	 torch.Size([10000, 64, 55, 55]) 	 computation time: 2.232000 s
MaxPool2d output shape:	 torch.Size([10000, 64, 27, 27]) 	 computation time: 3.585000 s
Conv2d output shape:	 torch.Size([10000, 192, 27, 27]) 	 computation time: 32.452000 s
ReLU output shape:	 torch.Size([10000, 192, 27, 27]) 	 computation time: 0.204000 s
MaxPool2d output shape:	 torch.Size([10000, 192, 13, 13]) 	 computation time: 2.837000 s
Conv2d output shape:	 torch.Size([10000, 384, 13, 13]) 	 computation time: 6.940000 s
ReLU output shape:	 torch.Size([10000, 384, 13, 13]) 	 computation time: 0.405000 s
Conv2d output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 7.328000 s
ReLU output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 0.230000 s
Conv2d output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 13.279000 s
ReLU output shape:	 torch.Size([10000, 256, 13, 13])

In [8]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
)

print(len(model))

3


In [36]:
"""
    尝试分解每层的计算量和参数量
"""
import numpy as np
import time

x = torch.rand(size=(100,3,224,224))
x = x.to(device)
alexNet2 = alexnet(input_layer = 3,num_classes = 1000)
print(next(alexNet2.parameters()).device)

idx = 1
for layer in alexNet2:
    start_time = int(round(time.time() * 1000))
    x = layer(x)
    end_time = int(round(time.time() * 1000))
    # print(x.device)

    # 计算分割点 中间传输占用大小为多少m  主要与网络传输时延相关
    total_num = 1
    for num in x.shape:
        total_num *= num
    type_size = 32
    size = total_num * type_size / 1000 /1000

    print("------------------------------------------------------------------")
    print(f'{idx}-{layer.__class__.__name__} computation time: {(end_time - start_time)/1000 :>3f} s\n'
          f'output shape: {x.shape} \t transport_num:{total_num} \t transport_size:{size}M')

    # 计算各层的结构所包含的参数量 主要与计算时延相关
    # para = parameters.numel()
    for name,parameters in layer.named_parameters():
        print(f"{name}  :  parameters size {parameters.size()} \t parameters number {parameters.numel()}")
    idx += 1

cpu
------------------------------------------------------------------
1-Conv2d computation time: 0.345000 s
output shape: torch.Size([100, 64, 55, 55]) 	 transport_num:19360000 	 transport_size:619.52M
weight  :  parameters size torch.Size([64, 3, 11, 11]) 	 parameters number 23232
bias  :  parameters size torch.Size([64]) 	 parameters number 64
------------------------------------------------------------------
2-ReLU computation time: 0.003000 s
output shape: torch.Size([100, 64, 55, 55]) 	 transport_num:19360000 	 transport_size:619.52M
------------------------------------------------------------------
3-MaxPool2d computation time: 0.020000 s
output shape: torch.Size([100, 64, 27, 27]) 	 transport_num:4665600 	 transport_size:149.2992M
------------------------------------------------------------------
4-Conv2d computation time: 0.092000 s
output shape: torch.Size([100, 192, 27, 27]) 	 transport_num:13996800 	 transport_size:447.89759999999995M
weight  :  parameters size torch.Size([